In [ ]:
# Source code for RecSys Model 2 Deployment
# Project Name: One Stop Solution App

In [ ]:
!python --version

Python 3.11.11


In [ ]:
# Temporary solution for a bug in the implementation of the tfrs.layers.factorized_top_k module.
# https://github.com/tensorflow/recommenders/issues/712#issuecomment-2041163592

!pip uninstall tensorflow -y
!pip uninstall tensorflow-recommenders -y

import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1


In [ ]:
!pip install -q tensorflow==2.17
!pip install -q tensorflow-recommenders==0.7.3


!pip install -q scann==1.3.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 89.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
from itertools import islice

from google.colab import drive

import scann
import tensorflow as tf
import tensorflow_recommenders as tfrs

import geopy
from geopy.distance import geodesic

import firebase_admin
from firebase_admin import firestore, credentials

In [ ]:
print(np.__version__)

1.26.4


In [ ]:
#print(scann.__version__)
!pip show scann

Name: scann
Version: 1.3.4
Summary: Scalable Nearest Neighbor search library
Home-page: https://github.com/google-research/google-research/tree/master/scann
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, tensorflow
Required-by: 


In [ ]:
print(tf.__version__)

2.17.0


In [ ]:
print(tfrs.__version__)

v0.7.3


In [ ]:
print(geopy.__version__)

2.4.1


In [ ]:
print(firebase_admin.__version__)

6.6.0


In [ ]:
import tf_keras
print(tf_keras.__version__)

2.17.0


# Loading the Models

In [ ]:
def load_models():
  '''Take no inputs, loads and returns two tensorflow models.'''

  retrieval_model_path = '/content/drive/My Drive/Colab Notebooks/Saved Models/recsys_model_two_retrieval'
  ranking_model_path = '/content/drive/My Drive/Colab Notebooks/Saved Models/recsys_model_two_ranking'

  # Load the models
  retrieval_model = tf.saved_model.load(retrieval_model_path)
  ranking_model = tf.saved_model.load(ranking_model_path)

  return retrieval_model, ranking_model

# Retrieve Matching Employees

In [ ]:
def retrieve_employees(retrieval_model, customer_id):
  '''Take a tensoflow model named "retrieval_model" and
  a string "customer_id". Returns a list containing
  employee ids that was retrieved from the above model
  based on the specified "customer_id".'''

  # Pass a category name and get top recommendations
  scores, employee_ids = retrieval_model(tf.constant([customer_id]))

  return employee_ids[0].numpy().tolist()  # Convert EagerTensor to list

# Rank Matching Employees

In [ ]:
def rank_employees(ranking_model, customer_id, retrieved_employees):
  '''Take a tensorflow model named "ranking_model", a string "customer_id"
  and a list "retrieved_employees". Returns a dictionary containing the top 100 employee ids
  with their ratings that was ranked by the above model based on the "customer_id".'''

  ranked_employees = {}

  # giving a rating for each employee id based on the "customer_id"
  for employee_id in retrieved_employees:
    ranked_employees[employee_id] = ranking_model({
        "customer_id": np.array([customer_id]),
        "employee_id": np.array([employee_id])
    })

  # Convert the dictionary to remove tensors, keeping only the numpy values
  ranked_employees = {key.decode("utf-8"): value.numpy().flatten()[0] for key, value in ranked_employees.items()}

  # Sort the dictionary by values in descending order
  ranked_employees = dict(sorted(ranked_employees.items(), key=lambda item: item[1], reverse=True))

  # Get the first/top 100 employee ids
  return dict(islice(ranked_employees.items(), 100))

# Get Data From Firestore

In [ ]:
def get_firestore_data(ranked_employees, customer_location, firestore_client):
  '''Take a dictionary named "ranked_employees", a tuple named "customer_location"
  and a client to interact with firestore api named "firestore_client". Returns a
  list of dictionaries containing the details of the 20 most nearest (within 20 km radius)
  active employees to the given "customer_location".'''

  recommended_employees = []

  collection_name = 'employees'


  for employee_id in ranked_employees.keys():

    # reference to the firestore document of the given employee id
    doc_ref = firestore_client.collection(collection_name).document(employee_id)

    # retrieving the firestore document of the given employee id
    doc = doc_ref.get()

    if doc.exists:
        employee_data = doc.to_dict()  # convert the document to a dictionary
        if employee_data['status'] == 'active': # take only active employees
          distance_from_customer = geodesic(customer_location, (employee_data['location'].latitude, employee_data['location'].longitude)).km
          if round(distance_from_customer) <= 20: # take only employees who are 20 km away or less from the given "customer_location"
            recommended_employees.append({
                'id': employee_id,
                'skills': ', '.join(employee_data['skills']),
                #'latitude': employee_data['location'].latitude,
                #'longitude': employee_data['location'].longitude,
                'address': employee_data['address'],
                'chargePerHour': employee_data['chargePerHour'],
                'email': employee_data['email'],
                'experience': employee_data['experience'],
                'firstName': employee_data['firstName'],
                'lastName': employee_data['lastName'],
                'numOfReviews': employee_data['numOfReviews'],
                'phone': employee_data['phone'],
                'profileImageUrl': employee_data['profileImageUrl'],
                'starCategoryCount': employee_data['starCategoryCount'],
                'stars': employee_data['stars'],
                'distanceFromCustomer': round(distance_from_customer, 2)
                })

  # Sorting the array ascendingly by 'distanceFromCustomer'
  recommended_employees = sorted(recommended_employees, key=lambda x: x['distanceFromCustomer'])

  # Return only the details of the nearest 20 employees
  return recommended_employees[:20]


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#JSON_KEY_FILE = '/content/drive/My Drive/firestore-446822-ac05de749f4a.json'

#JSON_KEY_FILE = "/content/drive/My Drive/service-pa-79ac5-0f0babe13506.json"

JSON_KEY_FILE = "/content/drive/My Drive/csg3101-service-providing-app-firebase-adminsdk-7xpfw-f39bc603db.json" # Path to Firebase Admin SDK credentials

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_KEY_FILE # Environment Variable with Credentials to Initialize Firebase Admin SDK (used as a backup)

# Initialize Firebase Admin SDK
cred = credentials.Certificate(JSON_KEY_FILE)
firebase_admin.initialize_app(cred)

In [ ]:
db = firestore.Client() # Client for interacting with Google Cloud Firestore API

In [ ]:
customer_id = "Ha3iJu77CxlrFm-vQRs_8g"
customer_location = (32.241638, -110.961819) # lat, long

In [ ]:
retrieval_model, ranking_model = load_models()

In [ ]:
retrieved_employees = retrieve_employees(retrieval_model, customer_id)

In [ ]:
ranked_employees = rank_employees(ranking_model, customer_id, retrieved_employees)

In [ ]:
recommended_employees = get_firestore_data(ranked_employees, customer_location, db)

In [ ]:
print(type(recommended_employees))

<class 'list'>


In [ ]:
print(len(recommended_employees))

20


In [ ]:
print(recommended_employees)

[{'id': '3Myk3oAOJq6Cps2SDYQZhA', 'skills': 'Food, Coffee & Tea, Nightlife, Lounges, Beauty & Spas, Cosmetics & Beauty Supply, Shopping, Tea Rooms, Bars', 'address': '943 E University Blvd, Ste 165', 'chargePerHour': 2609, 'email': '3myk3oaojq6cps2sdyqzha@employee.com', 'experience': 15, 'firstName': 'The Scented Leaf', 'lastName': '', 'numOfReviews': 184, 'phone': '072-895-6133', 'profileImageUrl': '', 'starCategoryCount': {'1 stars': 2, '5 stars': 147, '2 stars': 3, '4 stars': 31, '3 stars': 5}, 'stars': 4.5, 'distanceFromCustomer': '1.16 km'}, {'id': '0gdnntqYGYhUCTTf0a7Xcg', 'skills': 'Mexican, Vegetarian, Vegan, Restaurants', 'address': '402 E 4th St', 'chargePerHour': 2609, 'email': '0gdnntqygyhucttf0a7xcg@employee.com', 'experience': 5, 'firstName': 'Tumerico On 4th Ave', 'lastName': '', 'numOfReviews': 33, 'phone': '074-918-1890', 'profileImageUrl': '', 'starCategoryCount': {'1 stars': 0, '5 stars': 22, '2 stars': 1, '4 stars': 9, '3 stars': 1}, 'stars': 4.5, 'distanceFromCusto